In [ ]:
import sfacts as sf

Load the simulated metagenotype and filter

- positions by a minimum minor allele frequency
- samples by a minimum horizontal coverage (fraction of sites with counts)

In [ ]:
!sfacts filter_mgen \
    --min-minor-allele-freq 0.05 \
    --min-horizontal-cvrg 0.15 \
    --random-seed 0 \
    sim.mgen.nc sim.filt.mgen.nc

In [ ]:
!sfacts data_info sim.filt.mgen.nc

Now subsample 500 of the positions.

In [ ]:
mgen_raw = sf.data.Metagenotype.load('sim.mgen.nc')
mgen_filt = sf.data.Metagenotype.load('sim.filt.mgen.nc')
mgen_ss = sf.data.Metagenotype.load('sim.filt.mgen.nc')

print(mgen_raw.sizes)
print(mgen_filt.sizes)
print(mgen_ss.sizes)

We can see that this did not reduce the size of the data; that's because our simulation had
uniform coverage and plenty of polymorphism at each site.
(Real data will be filtered much more than this.)

Plotting the metagenotype again:

In [ ]:
sf.plot.plot_metagenotype(mgen_ss.to_world())

Now the fun part: fitting the StrainFacts model to these data.

Let's take a look at the details of the default StrainFacts model.

In [ ]:
!sfacts model_info default

We'll leave all of the hyperparameters set to their default values for this model.
In addition, we explicitly fit 15 strains (5 more than the simulation actually had),
and we set a random seed for reproducibility.

In [ ]:
!sfacts fit \
    --verbose \
    --num-strains 15 \
    --random-seed 0 \
    sim.filt.mgen.nc sim.filt.fit.world.nc

A model of this size on this dataset should fit relatively quickly (<2 minutes on my computer).

When run on the command-line, several pieces of information are printed to the screen,
thanks to the `--verbose` flag.

The result of this fit is a "world" with a point estimate for _all_ of the parameters.

Let's load this into Python and plot the inferred genotypes and relative abundances.

In [ ]:
fit = sf.data.World.load('sim.filt.fit.world.nc')

# Plot inferred relative abundances for each sample (the "community").
sf.plot.plot_community(
    fit,
    col_linkage_func=lambda w: w.metagenotype.linkage('sample'),
    row_linkage_func=lambda w: w.genotype.linkage('strain'),
)

# Plot the inferred genotype of the 10 simulated strains.
sf.plot.plot_genotype(
    fit,
    row_linkage_func=lambda w: w.genotype.linkage("strain"),
    col_linkage_func=lambda w: w.metagenotype.linkage("position"),
)

Finally, we'll dump the relative abundance and genotype inferences out to TSV files,
which could then be used by downstream tools.

Note that the genotypes for each strain in each position are encoded as a float,
where 0.0 means entirely reference and 1.0 means entirely alternative allele.

In [ ]:
!sfacts dump sim.filt.fit.refit.world.nc \
    --genotype sim.filt.fit.refit.geno.tsv \
    --community sim.filt.fit.refit.comm.tsv

In [ ]:
!head sim.filt.fit.refit.geno.tsv sim.filt.fit.refit.comm.tsv

In the next example, we'll compare this fit to the simulated ground-truth
in order to evaluate our performance.